In [ ]:
import numpy as np
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# K-fold cross validation wrapped into Keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

Running a  model can give different result on consequtive runs. K_fold cross validation gives a better validation of a model's performanace.

The data set is split in to ten (for example) sets. Each set is split into training and testing. The evarage result of the ten test sets are used to validate the performance of the model.

## Data preprocessing

In [ ]:
# import data
ds = pd.read_csv('data/Churn_Modelling.csv')
ds.head()

In [ ]:
# pick out colums to be used in model
X = ds.iloc[:, 3:13].values
y = ds.iloc[:, 13].values

In [ ]:
# encode categorical data
le_country = LabelEncoder()
le_gender = LabelEncoder()
X[:, 1 ] = le_country.fit_transform(X[:, 1])
X[:, 2 ] = le_gender.fit_transform(X[:, 2])

In [ ]:
# encode categorical data
ohe = OneHotEncoder(categorical_features=[1])
X = ohe.fit_transform(X).toarray()
X = X[:, 1:]

In [ ]:
# split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# feature scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Create ANN model

In [ ]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=11))
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

In [ ]:
# Classifier trained with k-fold cross validation
classifier = KerasClassifier(build_fn=build_classifier, batch_size=10, epochs=10)
accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10)

## Evaluate performance of model

In [ ]:
mean = accuracies.mean()
variance = accuracies.std()